<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/DBD_Charge_Speed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Simulation parameters
f = 10000  # Frequency (Hz)
E_rms = np.linspace(500, 5000, 10) / (np.sqrt(2) * np.pi * 0.1)  # Root mean square electric field (V/m)
L = 1  # Length of discharge (m)
d = 0.1  # Distance between electrodes (m)
W = 0.1  # Width of electrodes (m)
A = L * W  # Surface area of electrodes (m^2)
mu_0 = 4 * np.pi * 10**(-7)  # Permeability of free space (H/m)
epsilon_0 = 8.854 * 10**(-12)  # Permittivity of free space (F/m)
epsilon_r = 4  # Relative permittivity of dielectric material
sigma = 10**(-5)  # Conductivity of dielectric material (S/m)
ec = 1.6e-19  # Elementary charge
V_0 = 5000
N = 1000  # Number of points in space and time

def electric_field(x, t):
    if 0 <= x <= d:
        E_field = E * np.sin(2 * np.pi * f * t + 2 * np.pi * x / d)
    elif d < x <= L:
        E_field = E * np.sin(2 * np.pi * f * t + 2 * np.pi * (L - x) / d)
    else:
        E_field = 0
    return E_field

def charge_density(x, t):
    return sigma * electric_field(x, t) / (2 * epsilon_0 * epsilon_r)

# Initial conditions
y0 = [0, 0, 1e15, 1e13] # Initial values for V, Q, ne, and ni


t_f = 1 / f # Duration of discharge (s)

# Discretize space and time
x = np.linspace(0, L, N)
t = np.linspace(0, t_f, N)

def dbd_ode(t, y, x, E):
    V, Q, ne, ni = y
    E_field = electric_field(x, t)
    dVdx = -charge_density(x, t) / epsilon_0
    dQdx = sigma * A * E_field

    # Kinetic reactions due to electron impact
    alpha = 1.5e-17  # Ionization rate coefficient (m^3/s)
    beta = 1.5e-3  # Recombination rate coefficient (m^3/s)
    gamma = 1  # Secondary electron emission coefficient
    E_th = 30  # Threshold energy for secondary electron emission (eV)

# Secondary electron emission
    E_k = ec * (V - V_0)  # Kinetic energy of primary electrons (J)

    if V < 1e-10:
        n_e_se = 0
    else:
        n_e_se = gamma * ne * np.exp(-E_k / (ec * V)) * np.heaviside(E_k - E_th, 0.5)

    ne = (Q + n_e_se * ec * A * L) / (epsilon_0 * L)
    ni = ne / 100  # Ion density assuming ionization level of 1%
# Initialize the solution array
sol = np.empty((N, N, len(E_rms)))

for idx, E in enumerate(E_rms):
    for i in range(N):
        for j in range(N):
            result = solve_ivp(dbd_ode, (t[0], t[-1]), y0, args=(x[j], E), t_eval=[t[i]])
            sol[i, j, idx] = result.y[:, 0]


# Compute the gradient and J
J = -sigma * np.gradient(sol[:, 0], axis=0)

# Total charge deposited on electrodes
Q_total = []
for Q in sol[:, 1]:
    if Q >= 0:
        Q_total.append(Q)
    else:
        Q_total.append(0)

# Root mean square total charge deposited on electrodes
Q_rms = np.sqrt(np.mean(np.array(Q_total)**2))

# Root mean square gas speed
ne = Q_total[-1] / (epsilon_0 * A * L)  # Electron density (m^-3)
ni = ne / 100  # Ion density assuming ionization level of 1%
n = ne + ni  # Total plasma density
v_rms = (Q_rms / (ec * A)) * (mu_0 / (2 * np.pi)) * np.log((2 * L) / d) / (n + 1e-15)

# Plot current density versus time
plt.plot(t, sol[:, -1], label=f'{E:.2f} kV/cm')

# Add legend and labels to plot
plt.legend(title='Electric field')
plt.xlabel('Time (s)')
plt.ylabel('Current density (A/m^2)')
plt.show()

for idx, E in enumerate(E_rms):
   v_gas = []
   for Q in Q_total:
      ne = Q / (epsilon_0 * A * L) # Electron density (m^-3)
      ni = ne / 100 # Ion density assuming ionization level of 1%
      n = ne + ni # Total plasma density
      v = (Q / (ec * A)) * (mu_0 / (2 * np.pi)) * np.log((2 * L) / d) / (n + 1e-15)
      v_gas.append(v)

   # Plot gas speed versus time
   plt.plot(t, v_gas, label=f'{E:.2f} kV/cm')

# Add legend and labels to plot
plt.legend(title='Electric field')
plt.xlabel('Time (s)')
plt.ylabel('Gas speed (m/s)')
plt.show()

# Plot total charge deposited versus time
plt.figure()
for idx, E in enumerate(E_rms):
   Q_total = []
   for i in range(N):
      sol = odeint(dbd_ode, y0, x, args=(t[i], E))
      Q = sol[:, 1]
      Q = Q[Q >= 0]
      Q_total.append(np.sum(Q))

   # Total charge deposited on electrodes
   Q_total = []
   for Q in sol[:, 1]:
      if Q >= 0:
         Q_total.append(Q)
      else:
         Q_total.append(0)

   # Root mean square total charge deposited on electrodes
   Q_rms = np.sqrt(np.mean(np.array(Q_total)**2))

   # Root mean square gas speed
   ne = Q_total[-1] / (epsilon_0 * A * L) # Electron density (m^-3)
   ni = ne / 100 # Ion density assuming ionization level of 1%
   n = ne + ni

#   Compute root mean square gas speed
v_rms = (Q_rms / (ec * A)) * (mu_0 / (2 * np.pi)) * np.log((2 * L) / d) / (n + 1e-15)

# Plot current density versus time
plt.plot(t, sol[:, -1], label=f'{E:.2f} kV/cm')

# Add legend and labels to plot
plt.legend(title='Electric field')
plt.xlabel('Time (s)')
plt.ylabel('Current density (A/m^2)')
plt.show()

# Plot gas speed versus time
v_gas = []
for Q in Q_total:
    ne = Q / (epsilon_0 * A * L) # Electron density (m^-3)
    ni = ne / 100 # Ion density assuming ionization level of 1%
    n = ne + ni # Total plasma density
    v = (Q / (ec * A)) * (mu_0 / (2 * np.pi)) * np.log((2 * L) / d) / (n + 1e-15)
    v_gas.append(v)
    plt.plot(t, v_gas, label=f'{E:.2f} kV/cm')

# Add legend and labels to plot
plt.legend(title='Electric field')
plt.xlabel('Time (s)')
plt.ylabel('Gas speed (m/s)')
plt.show()

# Plot total charge deposited versus time
Q_total = []
for i in range(len(sol)):
    Q_total.append(np.sum(sol[i, 1]))
    plt.plot(t, Q_total, label=f'{E:.2f} kV/cm')

# Add legend and labels to plot
plt.legend(title='Electric field')
plt.xlabel('Time (s)')
plt.ylabel('Total charge deposited (C/m)')
plt.show()
